In [ ]:
import mmcv
from mmcv import Config
from mmdet.apis import set_random_seed

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


We need the train, valid, and test datasets in the COCO format saved in JSON files.

In [ ]:
!cp "/content/drive/My Drive/pets-train-coco-format.json" pets-train-coco-format.json

In [ ]:
!cp "/content/drive/My Drive/pets-test-coco-format.json" pets-test-coco-format.json

We need to specify the checkpoint from which the training will be started.

In [ ]:
!mkdir checkpoints
!wget -c https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
      -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

--2021-04-12 08:43:34--  https://open-mmlab.s3.ap-northeast-2.amazonaws.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
Resolving open-mmlab.s3.ap-northeast-2.amazonaws.com (open-mmlab.s3.ap-northeast-2.amazonaws.com)... 52.219.58.3
Connecting to open-mmlab.s3.ap-northeast-2.amazonaws.com (open-mmlab.s3.ap-northeast-2.amazonaws.com)|52.219.58.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177867103 (170M) [application/x-www-form-urlencoded]
Saving to: ‘checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth’

checkpoints/mask_rc 100%[===================>] 169.63M  15.4MB/s    in 12s     

2021-04-12 08:43:47 (14.3 MB/s) - ‘checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth’ saved [177867103/177867103]



In [ ]:
checkpoint = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

To use CometML we need to know where the Hook is saved. 

In [ ]:
%%writefile mmdet/core/utils/comet_logger_hook.py
from mmcv.runner import HOOKS, master_only
from mmcv.runner.hooks import LoggerHook


@HOOKS.register_module()
class CometMLLoggerHook(LoggerHook):

    def __init__(self,
                 project_name=None,
                 hyper_params=None,
                 import_comet=True,
                 interval=10,
                 ignore_last=True,
                 reset_flag=True,
                 by_epoch=True,
                 api_key=None):
        """Class to log metrics to Comet ML.
        It requires `comet_ml` to be installed.
        Args:
            project_name (str, optional):
                Send your experiment to a specific project. 
                Otherwise will be sent to Uncategorized Experiments. 
                If project name does not already exists Comet.ml will create 
                a new project.
            hyper_params (dict, optional): Logs a dictionary 
                (or dictionary-like object) of multiple parameters.
            import_comet (bool optional): Whether to import comet_ml before run.
                WARNING: Comet ML have to be imported before sklearn and torch,
                or COMET_DISABLE_AUTO_LOGGING have to be set in the environment.
            interval (int): Logging interval (every k iterations).
            ignore_last (bool): Ignore the log of last iterations in each epoch
                if less than `interval`.
            reset_flag (bool): Whether to clear the output buffer after logging
            by_epoch (bool): Whether EpochBasedRunner is used.
        """
        super(CometMLLoggerHook, self).__init__(interval, ignore_last,
                                                reset_flag, by_epoch)
        self._import_comet = import_comet
        if import_comet:
            self.import_comet()
        self.project_name = project_name
        self.hyper_params = hyper_params
        self._api_key = api_key

    def import_comet(self):
        try:
            import comet_ml
        except ImportError:
            raise ImportError(
                'Please run "pip install comet_ml" to install Comet ML')
        self.comet_ml = comet_ml

    @master_only
    def before_run(self, runner):
        if self._import_comet:
            self.experiment = self.comet_ml.Experiment(
                api_key=self._api_key,
                project_name=self.project_name,
            )
        else:
            self.experiment = comet_ml.Experiment(
                api_key=self._api_key,
                project_name=self.project_name,
            )
        if self.hyper_params is not None:
            self.experiment.log_parameters(self.hyper_params)

    @master_only
    def log(self, runner):
        tags = self.get_loggable_tags(runner, allow_text=True)
        for tag, val in tags.items():
            self.experiment.log_metric(name=tag,
                                       value=val,
                                       step=self.get_iter(runner),
                                       epoch=self.get_epoch(runner))

    @master_only
    def after_run(self, runner):
        self.experiment.end()

Writing mmdet/core/utils/comet_logger_hook.py


The easiest way to create a config is to modify an existing one.

In [ ]:
_base_ = './configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py'

In [ ]:
cfg = Config.fromfile(_base_)

In [ ]:
classes = (
    'Abyssinian',
    'american_bulldog',
    'american_pit_bull_terrier',
    'basset_hound',
    'beagle',
    'Bengal',
    'Birman',
    'Bombay',
    'boxer',
    'British_Shorthair',
    'chihuahua',
    'Egyptian_Mau',
    'english_cocker_spaniel',
    'english_setter',
    'german_shorthaired',
    'great_pyrenees',
    'havanese',
    'japanese_chin',
    'keeshond',
    'leonberger',
    'Maine_Coon',
    'miniature_pinscher',
    'newfoundland',
    'Persian',
    'pomeranian',
    'pug',
    'Ragdoll',
    'Russian_Blue',
    'saint_bernard',
    'samoyed',
    'scottish_terrier',
    'shiba_inu',
    'Siamese',
    'Sphynx',
    'staffordshire_bull_terrier',
    'wheaten_terrier',
    'yorkshire_terrier',
)

In [ ]:
# Changing number of classes
cfg.model.roi_head.bbox_head.num_classes = len(classes)
cfg.model.roi_head.mask_head.num_classes = len(classes)

# Folder in which train images are store.
cfg.data.train.img_prefix = 'images/'
cfg.data.train.classes = classes
# Path to train dataset in the COCO format.
cfg.data.train.ann_file = 'pets-train-coco-format.json'

# Folder in which val images are store.
cfg.data.val.img_prefix = 'images/'
cfg.data.val.classes = classes
# Path to val dataset in the COCO format.
cfg.data.val.ann_file = 'pets-test-coco-format.json'

# Folder in which test images are store.
cfg.data.test.img_prefix = 'images/'
cfg.data.test.classes = classes
# Path to test dataset in the COCO format.
cfg.data.test.ann_file = 'pets-test-coco-format.json'

# Path to the checkpoint from which training will be started.
cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# Logging interval
cfg.log_config.interval = 25

# Setting logging hooks, ee can set specific params, for example:
#          dict(type='CometMLLoggerHook', api_key='your_api_key')
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='CometMLLoggerHook')
]

# Path to CometML Hook file.
cfg.custom_imports = dict(imports=['mmdet.core.utils.comet_logger_hook'],
                          allow_failed_imports=False)

In [ ]:
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='MaskRCNN',
    pretrained='open-mmlab://detectron2/resnet50_caffe',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_

Finally, we have to save the config file in the .py file.

In [ ]:
cfg.dump('mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_pets.py')